# 03 · Tokenization & Packing

Tokenize canonical BHC records using Unsloth's 4-bit Llama tokenizer and pack sequences.

In [ ]:
# Persistent Drive + run mode setup
import os
import sys
from pathlib import Path

try:
    from google.colab import drive  # type: ignore
    DRIVE_MOUNT = Path('/content/drive')
    if not DRIVE_MOUNT.exists():
        drive.mount('/content/drive')
except Exception as exc:  # pragma: no cover
    print(f'Colab drive mount skipped: {exc}')

if Path('/content/drive').exists():
    DRIVE_ROOT = Path('/content/drive/MyDrive').resolve()
else:
    DRIVE_ROOT = Path.home().resolve()

PROJECT_ROOT = DRIVE_ROOT / 'secure-llm-mia'
if not PROJECT_ROOT.exists():
    raise FileNotFoundError('Run 00_colab_setup.ipynb first to clone the repo on Drive.')

if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

os.environ['SECURE_LLM_MIA_ROOT'] = str(PROJECT_ROOT)
os.chdir(PROJECT_ROOT)

from src.utils.runtime import current_run_mode

RUN_MODE = current_run_mode()
print('PROJECT_ROOT:', PROJECT_ROOT)
print('Active run mode:', RUN_MODE.name, '-', RUN_MODE.description)

DATA_ROOT = PROJECT_ROOT / 'data'
ARTIFACTS_DIR = PROJECT_ROOT / 'artifacts'
CHECKPOINT_ROOT = PROJECT_ROOT / 'checkpoints'
for path in (DATA_ROOT, ARTIFACTS_DIR, CHECKPOINT_ROOT):
    path.mkdir(parents=True, exist_ok=True)

BHC_DATA_DIR = DRIVE_ROOT / 'mimic-iv-bhc'
BHC_DATA_DIR.mkdir(parents=True, exist_ok=True)
BHC_CSV_PATH = BHC_DATA_DIR / 'mimic-iv-bhc.csv'
print('BHC CSV path:', BHC_CSV_PATH)


In [ ]:
import numpy as np

from unsloth import FastLanguageModel

from src.data.loaders import load_canonical

CANONICAL_PATH = ARTIFACTS_DIR / f'canonical_bhc_{RUN_MODE.name}.parquet'
df = load_canonical(CANONICAL_PATH)
if df.empty:
    raise FileNotFoundError('Canonical data missing. Run notebook 01 first.')


In [ ]:
MODEL_NAME = os.getenv('UNSLOTH_MODEL_NAME', 'unsloth/Meta-Llama-3.1-8B-bnb-4bit')
MAX_SEQ_LENGTH = 4096
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQ_LENGTH,
    load_in_4bit=True,
)
FastLanguageModel.for_inference(model)
print('Tokenizer eos_token:', tokenizer.eos_token)


In [ ]:
def pack_sequences(token_lists, max_length=4096):
    packed_batches = []
    current = []
    length = 0
    for tokens in token_lists:
        if length + len(tokens) > max_length:
            packed_batches.append(current)
            current = []
            length = 0
        current.extend(tokens)
        length += len(tokens)
    if current:
        packed_batches.append(current)
    return packed_batches

input_ids = tokenizer(df['text'].tolist(), return_attention_mask=False)['input_ids']
packed = pack_sequences(input_ids, max_length=MAX_SEQ_LENGTH)
print(f'Created {len(packed)} packed sequences for run mode {RUN_MODE.name}.')


💾 Export packed shards to Drive (Arrow/Parquet) once dataset access is configured.